In [68]:
import sys

sys.path.append("../")

import numpy as np
import pandas_ta as ta
import joblib 

from terminal.Extensions.MT5Ext import MT5Ext

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [69]:
ticks = np.load("./data/ticks_WIN$N_2024_6.npz", allow_pickle=True)["ticks"]

In [70]:
data = MT5Ext.create_ohlc_from_ticks(ticks, '5s')

In [71]:
# Cálculo de indicadores
data.ta.macd(append=True)
data.dropna(inplace=True)
data

,open,high,low,close,tick_volume,real_volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
time_msc,,,,,,,,,
2024-06-03 09:03:30,122550.0,122580.0,122540.0,122575.0,1052,2791,-58.313419,2.852300,-61.165719
2024-06-03 09:03:35,122575.0,122575.0,122540.0,122550.0,792,1902,-57.015468,3.320200,-60.335669
2024-06-03 09:03:40,122550.0,122575.0,122545.0,122575.0,855,2203,-53.354505,5.584931,-58.939436
2024-06-03 09:03:45,122575.0,122590.0,122565.0,122570.0,1341,3026,-50.277060,6.929900,-57.206961
2024-06-03 09:03:50,122565.0,122570.0,122530.0,122545.0,1467,5102,-49.287301,6.335728,-55.623029
...,...,...,...,...,...,...,...,...,...
2024-06-28 18:24:40,125285.0,125285.0,125280.0,125280.0,8,16,-3.713811,-2.374789,-1.339023
2024-06-28 18:24:45,125275.0,125280.0,125270.0,125270.0,12,91,-4.756040,-2.733614,-2.022426
2024-06-28 18:24:50,125270.0,125295.0,125270.0,125280.0,23,62,-4.720680,-2.158603,-2.562077


In [62]:
# Pré-processamento de dados
data['change'] = data['close'].pct_change()
data['target'] = np.where(data['change'] > 0, 1, 0)

preprocessing = data.copy()
preprocessing['target'] = preprocessing['target'].shift(-1)
preprocessing.dropna(inplace=True)
preprocessing

,open,high,low,close,tick_volume,real_volume,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,change,target
time_msc,,,,,,,,,,,
2024-06-03 09:03:35,122575.0,122575.0,122540.0,122550.0,792,1902,-57.015468,3.320200,-60.335669,-0.000204,1.0
2024-06-03 09:03:40,122550.0,122575.0,122545.0,122575.0,855,2203,-53.354505,5.584931,-58.939436,0.000204,0.0
2024-06-03 09:03:45,122575.0,122590.0,122565.0,122570.0,1341,3026,-50.277060,6.929900,-57.206961,-0.000041,0.0
2024-06-03 09:03:50,122565.0,122570.0,122530.0,122545.0,1467,5102,-49.287301,6.335728,-55.623029,-0.000204,0.0
2024-06-03 09:03:55,122545.0,122560.0,122530.0,122535.0,1192,3067,-48.747891,5.500110,-54.248001,-0.000082,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2024-06-28 18:24:35,125285.0,125285.0,125280.0,125280.0,7,33,-3.347712,-2.602386,-0.745326,0.000000,0.0
2024-06-28 18:24:40,125285.0,125285.0,125280.0,125280.0,8,16,-3.713811,-2.374789,-1.339023,0.000000,0.0
2024-06-28 18:24:45,125275.0,125280.0,125270.0,125270.0,12,91,-4.756040,-2.733614,-2.022426,-0.000080,1.0


In [63]:
# Criar características e rótulos
feature_columns = ['open', 'close', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9']
features = preprocessing[feature_columns]
target = preprocessing['target']

In [64]:
# Dividir em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [65]:
# Treinar o modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [66]:
# Avaliar o modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'accuracy: {accuracy:.2f}')

accuracy: 0.56


In [67]:
# save
joblib.dump(model, "../models/test.joblib")

['../models/test.joblib']